In [2]:
import pyprojroot

def make_dir_function(dir_name):
    def dir_function(*args):
        return pyprojroot.here().joinpath(dir_name, *args)

    return dir_function

In [3]:
data_dir = make_dir_function("Data")
data_dir("raw", "songs_normalize.csv")

WindowsPath('j:/DOCUEMENTOS/PROYECTOS PERSONALES/SPOTIFY/project/Data/raw/songs_normalize.csv')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics 
import seaborn as sns

In [5]:
df=pd.read_csv(data_dir("raw", "songs_normalize.csv"))

## ML model simple linear regression

In [6]:
data = df.copy()

In [7]:
data.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


In [8]:
X = data['energy'].values.reshape(-1, 1)
y = data['loudness'].values.reshape(-1, 1)

sc_x = StandardScaler()
sc_y = StandardScaler()

X_std = sc_x.fit_transform(X)
y_std = sc_y.fit_transform(y)

slr = LinearRegression()
slr.fit(X_std, y_std)

LinearRegression()

LinearRegression()

## Analysis of results

In [16]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter(
    df,
    x=X_std.flatten(),
    y=y_std.flatten(),
    opacity=0.2,
    template='plotly_dark',
    title='<b>Energy versus Loudness & Linear Regression</b>'
)

fig.add_trace(go.Scatter(
    x=X_std.flatten(),
    y=slr.predict(X_std).flatten(),
    mode='lines',
    name='Linear Regression',
    line=dict(color='red', width=2.5)  # Ajustar el grosor de la línea aquí
))

fig.update_layout(
    xaxis=dict(title='Energy'),
    yaxis=dict(title='Loudness'),
    hovermode='x', 
    title_x=0.5)
fig.show()

## Model test

- Values of loudness typically range between -60 and 0 db

In [40]:
energy_song = 0.8
energy_song_std = sc_x.transform(np.array([energy_song]).reshape(-1,1))
slr.predict(energy_song_std)
print('The loudness of the song depending on the energy value is: ', sc_y.inverse_transform(slr.predict(energy_song_std)), 'dB')

The loudness of the song depending on the energy value is:  [[-4.85619525]] dB
